<a href="https://colab.research.google.com/github/francis-100/ai_bootcamp/blob/main/common_diseases.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install streamlit
!pip install pyngrok==4.1.1
!pip install streamlit_option_menu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 44.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-4.1.1-py3-none-any.whl size=15963 sha256=89c6665e9ee7e9f8e47755feefec7abe376815f06089d82f49838b6027136ecd
  Stored in directory: /root/.cache/pip/wheels/4c/7c/4c/632fba2ea8e88d8890102eb07bc922e1ca8fa14db5902c91a8
Successfully built pyngrok
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 822.1/822.1 kB 8.6 MB/s eta 0:00:00


In [68]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.svm import SVR
from sklearn.metrics import accuracy_score
from pyngrok import ngrok
import streamlit as st
import pickle
import warnings

In [69]:
# To ignore all warnings
warnings.filterwarnings("ignore")

In [60]:
# loading the diabetes dataset to a pandas DataFrame
# diabetes_dataset = pd.read_csv('/content/diabetes.csv')
# diseases_dataset = pd.read_csv('common_diseases.csv')
# df_train = pd.read_csv('Training.csv')
# df_test = pd.read_csv('Training.csv')
df_main = pd.read_csv('diseases_training.csv')

In [94]:
df_main['diagnosis'] = df_main['diagnosis'].round().astype(int)

In [95]:
# Create a new DataFrame with only the selected symptoms
selected_symptoms = ["itching", "skin_rash", "continuous_sneezing", "chills", "joint_pain", "vomiting",
                     "fatigue", "weight_loss", "cough", "high_fever", "headache", "yellowish_skin","prognosis","diagnosis"]

df = df_main[selected_symptoms]

In [96]:
df['diagnosis'] = df['diagnosis'].round().astype(int)

In [97]:
# printing the first 5 rows of the dataset
# diabetes_dataset.head()
# diseases_dataset.head()
df.head()

,itching,skin_rash,continuous_sneezing,chills,joint_pain,vomiting,fatigue,weight_loss,cough,high_fever,headache,yellowish_skin,prognosis,diagnosis
0,1,1,0,0,0,0,0,0,0,0,0,0,Fungal infection,1
1,0,1,0,0,0,0,0,0,0,0,0,0,Fungal infection,1
2,1,0,0,0,0,0,0,0,0,0,0,0,Fungal infection,1
3,1,1,0,0,0,0,0,0,0,0,0,0,Fungal infection,1
4,1,1,0,0,0,0,0,0,0,0,0,0,Fungal infection,1


In [98]:
df.tail()

,itching,skin_rash,continuous_sneezing,chills,joint_pain,vomiting,fatigue,weight_loss,cough,high_fever,headache,yellowish_skin,prognosis,diagnosis
4915,0,0,0,0,0,1,0,0,0,0,1,0,(vertigo) Paroymsal Positional Vertigo,37
4916,0,1,0,0,0,0,0,0,0,0,0,0,Acne,38
4917,0,0,0,0,0,0,0,0,0,0,0,0,Urinary tract infection,39
4918,0,1,0,0,1,0,0,0,0,0,0,0,Psoriasis,40
4919,0,1,0,0,0,0,0,0,0,1,0,0,Impetigo,41


In [73]:
# number of rows and Columns in this dataset
# diabetes_dataset.shape
# diseases_dataset.shape
df.shape

(4920, 13)

In [99]:
# getting the statistical measures of the data
# diabetes_dataset.describe()
# diseases_dataset.describe()
df.describe()

,itching,skin_rash,continuous_sneezing,chills,joint_pain,vomiting,fatigue,weight_loss,cough,high_fever,headache,yellowish_skin,diagnosis
count,4920.000000,4920.000000,4920.000000,4920.000000,4920.000000,4920.000000,4920.000000,4920.000000,4920.000000,4920.000000,4920.000000,4920.000000,4920.000000
mean,0.137805,0.159756,0.045122,0.162195,0.139024,0.389024,0.392683,0.092683,0.114634,0.276829,0.230488,0.185366,20.999797
std,0.344730,0.366417,0.207593,0.368667,0.346007,0.487578,0.488397,0.290017,0.318612,0.447477,0.421188,0.388634,11.833680
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,11.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,21.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,31.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,41.000000


In [36]:
# df_train.drop('Unnamed: 133', axis=1, inplace=True)

In [78]:
df['prognosis'].unique()

array(['Fungal infection', 'Allergy', 'GERD', 'Chronic cholestasis',
       'Drug Reaction', 'Peptic ulcer diseae', 'AIDS', 'Diabetes',
       'Gastroenteritis', 'Bronchial Asthma', 'Hypertension', 'Migraine',
       'Cervical spondylosis', 'Paralysis (brain hemorrhage)', 'Jaundice',
       'Malaria', 'Chicken pox', 'Dengue', 'Typhoid', 'hepatitis A',
       'Hepatitis B', 'Hepatitis C', 'Hepatitis D', 'Hepatitis E',
       'Alcoholic hepatitis', 'Tuberculosis', 'Common Cold', 'Pneumonia',
       'Dimorphic hemmorhoids(piles)', 'Heart attack', 'Varicose veins',
       'Hypothyroidism', 'Hyperthyroidism', 'Hypoglycemia',
       'Osteoarthristis', 'Arthritis',
       '(vertigo) Paroymsal  Positional Vertigo', 'Acne',
       'Urinary tract infection', 'Psoriasis', 'Impetigo'], dtype=object)

In [79]:
# diabetes_dataset['Outcome'].value_counts()
# diseases_dataset['Disease'].value_counts()
class_counts = df['prognosis'].value_counts()
class_counts

Fungal infection                           120
Hepatitis C                                120
Hepatitis E                                120
Alcoholic hepatitis                        120
Tuberculosis                               120
Common Cold                                120
Pneumonia                                  120
Dimorphic hemmorhoids(piles)               120
Heart attack                               120
Varicose veins                             120
Hypothyroidism                             120
Hyperthyroidism                            120
Hypoglycemia                               120
Osteoarthristis                            120
Arthritis                                  120
(vertigo) Paroymsal  Positional Vertigo    120
Acne                                       120
Urinary tract infection                    120
Psoriasis                                  120
Hepatitis D                                120
Hepatitis B                                120
Allergy      

In [80]:
# diabetes_dataset.groupby('Outcome').mean()
# diseases_dataset.groupby('Disease').mean()
df.groupby('prognosis').mean()

,itching,skin_rash,continuous_sneezing,chills,joint_pain,vomiting,fatigue,weight_loss,cough,high_fever,headache,yellowish_skin,diagnosis
prognosis,,,,,,,,,,,,,
(vertigo) Paroymsal Positional Vertigo,0.00,0.00,0.00,0.00,0.00,0.95,0.00,0.00,0.00,0.00,0.95,0.00,36.999908
AIDS,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.95,0.00,0.00,7.000321
Acne,0.00,0.95,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,37.998992
Alcoholic hepatitis,0.00,0.00,0.00,0.00,0.00,0.95,0.00,0.00,0.00,0.00,0.00,0.95,24.999725
Allergy,0.00,0.00,0.90,0.90,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2.001265
Arthritis,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,35.999633
Bronchial Asthma,0.00,0.00,0.00,0.00,0.00,0.00,0.90,0.00,0.90,0.95,0.00,0.00,10.000183
Cervical spondylosis,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,13.000092
Chicken pox,0.95,0.95,0.00,0.00,0.00,0.00,0.95,0.00,0.00,0.95,0.95,0.00,17.000275


In [101]:
# separating the data and labels
# X = diabetes_dataset.drop(columns = 'Outcome', axis=1)
# Y = diabetes_dataset['Outcome']
# X = diseases_dataset.drop(columns = 'Disease', axis=1)
# Y = diseases_dataset['Disease']
y = df['diagnosis']
X = df.drop(['diagnosis', 'prognosis'], axis=1)

In [91]:
# df['diagnosis'] = df['diagnosis'].round().astype(int)

In [102]:
print(X)

      itching  skin_rash  continuous_sneezing  chills  joint_pain  vomiting  \
0           1          1                    0       0           0         0   
1           0          1                    0       0           0         0   
2           1          0                    0       0           0         0   
3           1          1                    0       0           0         0   
4           1          1                    0       0           0         0   
...       ...        ...                  ...     ...         ...       ...   
4915        0          0                    0       0           0         1   
4916        0          1                    0       0           0         0   
4917        0          0                    0       0           0         0   
4918        0          1                    0       0           1         0   
4919        0          1                    0       0           0         0   

      fatigue  weight_loss  cough  high_fever  head

In [103]:
print(y)

0        1
1        1
2        1
3        1
4        1
        ..
4915    37
4916    38
4917    39
4918    40
4919    41
Name: diagnosis, Length: 4920, dtype: int64


In [104]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, stratify=y, random_state=2)

In [85]:
print(X.shape, X_train.shape, X_test.shape)

(4920, 12) (3936, 12) (984, 12)


In [86]:
classifier = svm.SVC(kernel='linear')

In [40]:
# # Transform
# regressor = SVR()
# regressor.fit(X_train, y_train)

In [105]:
print(y_train.unique())  # Check unique values in y_train

[ 3  9  6 22 41 30 23 15 14 11 35 19 16 17 26  2 36 18 28  4  5 13  7 37
 29 31 25 20 38 40 10 32 34 33 21  1 12 27 24 39  8]


In [106]:
#training the support vector Machine Classifier
classifier.fit(X_train, y_train)

SVC(kernel='linear')

In [107]:
# accuracy score on the training data
X_train_prediction = classifier.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, y_train)

In [108]:
print('Accuracy score of the training data : ', training_data_accuracy)

Accuracy score of the training data :  0.7248475609756098


In [109]:
# accuracy score on the test data
X_test_prediction = classifier.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, y_test)

In [110]:
print('Accuracy score of the test data : ', test_data_accuracy)

Accuracy score of the test data :  0.709349593495935


In [111]:
import numpy as np

# Assuming you have a trained classifier, replace 'YourClassifier' with your actual classifier
# classifier = YourClassifier()

# Input data for 12 symptoms
input_data = (1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1)

# Convert the input data to a numpy array
input_data_as_numpy_array = np.asarray(input_data)

# Reshape the array as we are predicting for one instance
input_data_reshaped = input_data_as_numpy_array.reshape(1, -1)

# Assuming 'classifier' is a trained model
prediction = classifier.predict(input_data_reshaped)

diseases = [
    "Paroymsal Positional Vertigo", "AIDS", "Acne", "Alcoholic hepatitis", "Allergy", "Arthritis",
    "Bronchial Asthma", "Cervical spondylosis", "Chicken pox", "Chronic cholestasis", "Common Cold",
    "Dengue", "Diabetes", "Dimorphic hemmorhoids(piles)", "Drug Reaction", "Fungal infection", "GERD",
    "Gastroenteritis", "Heart attack", "Hepatitis B", "Hepatitis C", "Hepatitis D", "Hepatitis E",
    "Hypertension", "Hyperthyroidism", "Hypoglycemia", "Hypothyroidism", "Impetigo", "Jaundice", "Malaria",
    "Migraine", "Osteoarthristis", "Paralysis (brain hemorrhage)", "Peptic ulcer disease", "Pneumonia",
    "Psoriasis", "Tuberculosis", "Typhoid", "Urinary tract infection", "Varicose veins", "Hepatitis A",
]

if 1 <= prediction[0] <= len(diseases):
    print(f'The patient is suffering from {diseases[prediction[0] - 1]}.')
else:
    print('The patient is suffering from some disease.')

The patient is suffering from Drug Reaction.


In [112]:
import pickle

In [113]:
# filename = 'diabetes_model.sav'
filename = 'diseases_model.sav'
pickle.dump(classifier, open(filename, 'wb'))

In [114]:
# loading the saved model
# loaded_model = pickle.load(open('diabetes_model.sav', 'rb'))
loaded_model = pickle.load(open('diseases_model.sav', 'rb'))

In [115]:
import numpy as np

# Assuming you have a trained classifier, replace 'YourClassifier' with your actual classifier
# classifier = YourClassifier()

# Input data for 12 symptoms
input_data = (1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0)

# Convert the input data to a numpy array
input_data_as_numpy_array = np.asarray(input_data)

# Reshape the array as we are predicting for one instance
input_data_reshaped = input_data_as_numpy_array.reshape(1, -1)

# Assuming 'classifier' is a trained model
prediction = classifier.predict(input_data_reshaped)

diseases = [
    "Paroymsal Positional Vertigo", "AIDS", "Acne", "Alcoholic hepatitis", "Allergy", "Arthritis",
    "Bronchial Asthma", "Cervical spondylosis", "Chicken pox", "Chronic cholestasis", "Common Cold",
    "Dengue", "Diabetes", "Dimorphic hemmorhoids(piles)", "Drug Reaction", "Fungal infection", "GERD",
    "Gastroenteritis", "Heart attack", "Hepatitis B", "Hepatitis C", "Hepatitis D", "Hepatitis E",
    "Hypertension", "Hyperthyroidism", "Hypoglycemia", "Hypothyroidism", "Impetigo", "Jaundice", "Malaria",
    "Migraine", "Osteoarthristis", "Paralysis (brain hemorrhage)", "Peptic ulcer disease", "Pneumonia",
    "Psoriasis", "Tuberculosis", "Typhoid", "Urinary tract infection", "Varicose veins", "Hepatitis A",
]

if 1 <= prediction[0] <= len(diseases):
    print(f'The patient is suffering from {diseases[prediction[0] - 1]}.')
else:
    print('The patient is suffering from some disease.')

The patient is suffering from GERD.


In [116]:
for column in X.columns:
  print(column)

itching
skin_rash
continuous_sneezing
chills
joint_pain
vomiting
fatigue
weight_loss
cough
high_fever
headache
yellowish_skin


In [54]:
!ngrok authtoken 2b2ZkpRhoJalycZbN3owwgNcxGy_wC76yMTvMEXk7r8Fjwwo

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [55]:
# Stop any existing ngrok tunnels (if any)
ngrok.kill()

In [58]:
# Set up the streamlit app
# !streamlit run app.py &>/dev/null&
!streamlit run --server.port 8501 app.py &



2024-01-19 09:55:28.362 Port 8501 is already in use


In [59]:
# Set up the ngrok tunnel
public_url = ngrok.connect(port='8501')
public_url

'http://acb2-35-204-241-131.ngrok-free.app'